# first look at user data

In [3]:
# setup
import os
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from both .env and shell
load_dotenv()  # This adds .env variables to os.environ

def create_engine_with_params(db_params: dict) -> create_engine:
    """Create SQLAlchemy engine using database parameters."""
    url = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
    return create_engine(url)

db_params = {
    "dbname": "postgres", 
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": "5432"  # Default port for PostgreSQL
}

engine = create_engine_with_params(db_params)

In [4]:
# recent records
df = pd.read_sql_query("SELECT * FROM events", con=engine)
df.sort_values(by="timestamp", ascending=False).head()

,event_id,timestamp,session_id,type,data,metadata
71,47c8ae19-e931-4eb1-9975-a0b256ff87da,2025-01-30 07:01:57.673085+00:00,ad740480-2ea6-464f-a8dc-f235fd8771f7,meme_created,{'prompt': 'Toddler is not impressed with this...,"{'environment': 'development', 'timestamp_utc'..."
70,1c61a7ba-7b94-435d-af16-4ec7e548d0ca,2025-01-30 07:01:36.183643+00:00,ad740480-2ea6-464f-a8dc-f235fd8771f7,meme_created,{'prompt': 'Toddler is not impressed with this...,"{'environment': 'development', 'timestamp_utc'..."
69,78b6ee9c-7cc4-4a4d-965e-50d1538bc72d,2025-01-30 07:01:12.527844+00:00,ad740480-2ea6-464f-a8dc-f235fd8771f7,meme_created,{'prompt': 'Toddler is not impressed with this...,"{'environment': 'development', 'timestamp_utc'..."
68,20807149-682b-4ff6-add7-f4f0cc7f039f,2025-01-30 07:01:02.015821+00:00,ad740480-2ea6-464f-a8dc-f235fd8771f7,meme_created,{'prompt': 'Toddler is not impressed with this...,"{'environment': 'development', 'timestamp_utc'..."
67,08a4f448-b127-4588-9e6e-43fd80022d7e,2025-01-30 07:00:56.291250+00:00,ad740480-2ea6-464f-a8dc-f235fd8771f7,meme_created,{'prompt': 'Toddler is not impressed with this...,"{'environment': 'development', 'timestamp_utc'..."


In [6]:
# logs can get better - i.e. differentiate first try from retries; encourage meme liked
df.type.value_counts()

type
meme_created       45
error              14
llm_response        5
meme_liked          3
test_event          2
meme_downloaded     2
startup             1
Name: count, dtype: int64

In [13]:
# we want to fix these logs such that we see what the api payload is - so we quickly know the captions
# might also make sense to add a "reasoning" field to the LLM output
df[df['data'].apply(lambda x: 'prompt' in x)]['data'].iloc[5]

{'prompt': 'Help me express the stress of trying to get a live demo working in very little time.',
 'image_url': 'https://i.imgflip.com/9iewoc.jpg',
 'template_id': '181913649'}

In [15]:
# First filter for rows with prompts, then extract the desired fields
meme_details = df['data'].apply(pd.Series)[['prompt', 'image_url', 'template_id']]

# Some rows might not have image_url or template_id (like error cases)
# so let's filter to only successful meme creations
created_memes = meme_details.dropna()

# Display the results
created_memes.head()

,prompt,image_url,template_id
14,Help me express the stress of trying to get a ...,https://i.imgflip.com/9iewoc.jpg,181913649
16,Help me express the stress of trying to get a ...,https://i.imgflip.com/9iewvf.jpg,181913649
17,Help me express the stress of trying to get a ...,https://i.imgflip.com/9iewwc.jpg,181913649
18,Help me express the stress of trying to get a ...,https://i.imgflip.com/9iewwn.jpg,181913649
20,I am trying to create a funny meme using AI an...,https://i.imgflip.com/9iexgl.jpg,181913649


In [18]:
# let's explore and annoted this first set using excel
# goal is to create a set of 20-30 example prompts that we can use to evaluate the model
# we can also annotate and some statistics like "is_drake" - but also off_optic, and with more insight to payload, is_json, is_complete 
created_memes.to_excel("../data/evals/created_memes.xlsx",sheet_name="raw_data")

# Creating tests

# Debugging errors
Helped picked up JSON errors

In [15]:

df.sort_values(by="timestamp", ascending=False).head().iloc[0]['data']

{'prompt': 'I am trying to demo this app and it is quite flaky i.e. errors quite often',
 'error_type': 'RuntimeError',
 'error_message': 'Failed after 10 attempts. Last error: Attempt 10 failed: Expecting value: line 1 column 1 (char 0)'}

In [6]:
df.sort_values(by="timestamp", ascending=False).head(10).iloc[5]

event_id                   d17555ad-f8e0-4759-b920-38878f323357
timestamp                      2025-01-30 02:05:21.262870+00:00
session_id                 fca15b75-dca1-4cad-9dba-78dbd15e34e3
type                                                      error
data          {'prompt': 'I want to make a funny meme and am...
metadata      {'environment': 'development', 'timestamp_utc'...
Name: 34, dtype: object

In [7]:
df.sort_values(by="timestamp", ascending=False).head(10).iloc[5]['data']

{'prompt': 'I want to make a funny meme and am using AI as a crutch',
 'error_type': 'RuntimeError',
 'error_message': 'Failed after 5 attempts. Last error: Attempt 5 failed: Expecting value: line 1 column 1 (char 0)'}

In [9]:
df.sort_values(by="timestamp", ascending=False).head(10).iloc[0]['data']

{'prompt': 'Ice cream is too cold and expensive. Grandpa yells at cloud.',
 'image_url': 'https://i.imgflip.com/9ifa2x.jpg',
 'template_id': '181913649'}